In [9]:
import sys
sys.path.append('..')
import constants.file_handler_constants as fh
import numpy as np
from packages.attraction.Attraction import *
from packages.restaurant.Restaurant import *
import os
import glob
import time
import pandas as pd
import numpy as np
from constants.attraction_constants import *
import re
import webbrowser
from selenium.webdriver import Remote, ChromeOptions
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [10]:
geo_code_df = pd.read_csv(fh.PATH_TO_GEOCODE)

In [11]:
url = "https://www.google.com/maps/search/?api=1&query=7.880209799999999,98.37213659999999?pi=1&query=7.880209799999999,98.37213659999999"
# URL provided by the user
url = "https://www.google.com/maps/search/?api=1&query=13.847111,100.6007"

# Extract the query part containing lat,long using regular expression
match = re.search(r"query=([-+]?\d*\.\d+|\d+),([-+]?\d*\.\d+|\d+)", url)

# Extract latitude and longitude if match is found
if match:
    latitude = match.group(1)
    longitude = match.group(2)
    print("lat, long -> ",latitude, " ", longitude)

lat, long ->  13.847111   100.6007


In [12]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
# options.add_argument('--version=128.0.6613.85')
sbr_connection = ChromiumRemoteConnection(os.environ["SBR_WS_ENDPOINT"], 'goog', 'chrome')
driver = webdriver.Remote(command_executor=sbr_connection, options=options)
print("driver ...", driver)
driver.get("https://www.wongnai.com/restaurants/2863371uJ-%E0%B9%84%E0%B8%AD%E0%B8%95%E0%B8%B4%E0%B8%A1%E0%B9%82%E0%B8%AD%E0%B9%88%E0%B8%87-%E0%B8%AB%E0%B8%B2%E0%B8%94%E0%B9%83%E0%B8%AB%E0%B8%8D%E0%B9%88?_st=cD0xO2I9Mjg2MzM3MTthZD10cnVlO3Q9MTcyNjI0MjIzNTg5MDtyaT0xWDdiNXlzWDZmQ2JsVzdPRHhqQzdmNHJJMVprNVg7aT0xWDcwWkllN2VJajZiM3RuU2djNnZhQUxJaklZRUI7d3JlZj1zcjs%3D")
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'li')))
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'li')))
all_meta_data = driver.find_elements(By.TAG_NAME, 'meta')
cnt_lat_long = 0
lat = 0
long = 0
lat_property = "place:location:latitude"
long_property = "place:location:longitude"
for cur_meta_data in all_meta_data:
    if(cnt_lat_long == 2):
        break
    cur_property = cur_meta_data.get_attribute('property')
    print("cur_prop --> ", cur_property)
    if(cur_property):
        if(cur_property == lat_property):
            print(cur_meta_data.get_attribute('content'))
            lat = float(cur_meta_data.get_attribute('content'))
            cnt_lat_long += 1
            continue
        elif(cur_property == long_property):
            print(cur_meta_data.get_attribute('content'))
            long = float(cur_meta_data.get_attribute('content'))
            cnt_lat_long += 1
            continue

print("lat : ", lat)
print("long : ", long)
time.sleep(10*800)
driver.close()

KeyboardInterrupt: 

In [4]:
x1 = "https://www.wongnai.com/restaurants/2669474sn-cafe-delight-phuket"
x2 = "https://www.wongnai.com/restaurants/2686171Bf-napa-massage-patong?_st=cD0yO2I9MjY4NjE3MTthZD1mYWxzZTt0PTE3MjYzMzQ1MDEwODI7cmk9MVg3YjYxdFkwRWVpVTJlQ2RpcjFQUThuR29ZUklUO2k9MVg3MFpMZjhKMEhaTFRoS1hiTWtzSTJXTDNIck4xO3dyZWY9c3I7"

a = x1
find_mark_Idx = a.find('?')
res_mark_Idx = find_mark_Idx if find_mark_Idx != -1 else len(a)
all_img_query = '%s/photos' % (a[:res_mark_Idx])
all_img_query

'https://www.wongnai.com/restaurants/2669474sn-cafe-delight-phuket/photos'

In [18]:
arr = np.arange(50)
arr = arr + 1
print(arr)
newarr = np.array_split(arr, 2)

print(newarr)


[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50]
[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25]), array([26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
       43, 44, 45, 46, 47, 48, 49, 50])]
